In [1]:
import nltk.corpus
import string
import re , math
from collections import Counter

In [56]:
#slang dict creation
f = open('/home/tanya/Desktop/major/after_phase1/analyse/classifiers/slang.txt', 'r')
slang_dict = {}
for line in f:
    line = line.strip()
    parts = [p.strip() for p in line.split("=")]
    slang_dict[parts[0].lower()] = (parts[1].lower())
f.close()
#print slang_dict

#creation of abusive words list
file1 = open('/home/tanya/Desktop/major/after_phase1/analyse/classifiers/abusive.txt')
abusive = list()
line = file1.readline()
while line:
    word = line.rstrip()
    abusive.append(word)
    line=file1.readline()
file1.close()

#creation of less offensive dict
file1 = open('/home/tanya/Desktop/major/after_phase1/analyse/classifiers/less_offensive.txt')
less_offensive = list()
line = file1.readline()
while line:
    word = line.rstrip()
    less_offensive.append(word)
    line=file1.readline()
file1.close()
#stop list
stop_list = nltk.corpus.stopwords.words('english')
stop_list.extend(string.punctuation)
stop_list.append('atuser')
stop_list.append('url')
stop_list.append('rt')

#phrase_dict
file1 = open('/home/tanya/Desktop/major/after_phase1/analyse/classifiers/phrase.txt')
phrase_list = list()
line = file1.readline()
while line:
    word = line.rstrip()
    phrase_list.append(word)
    line=file1.readline()
file1.close()

#adjective list
file1 = open('/home/tanya/Desktop/major/after_phase1/analyse/classifiers/adjectives.txt')
adj = list()
line = file1.readline()
while line:
    word = line.rstrip()
    adj.append(word)
    line=file1.readline()
file1.close()


In [ ]:
import os
import sys
os.chdir("/home/tanya/Desktop/major/spark")
os.curdir
if 'SPARK_HOME' not in os.environ:
    os.environ['SPARK_HOME']='/home/tanya/spark-2.0.0-bin-hadoop2.4'
SPARK_HOME=os.environ['SPARK_HOME']

sys.path.insert(0,os.path.join(SPARK_HOME,"python"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","pyspark.zip"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","py4j-0.10.1-src.zip"))

from pyspark import SparkContext
from pyspark import SparkConf

conf=SparkConf()
conf.set("spark.executor.memory","1g")
conf.set("spark.cores.max","1")

conf.setAppName("major")

sc = SparkContext('local',conf=conf)



In [6]:
from pyspark.sql import SQLContext
import enchant
import editdistance 
d = enchant.Dict("en_US")
sqlContext = SQLContext(sc)
import re ,string
def preprocess(x):
    l = x.split(',')
    if(len(l)>=4):
        #convert to lower case
        text = str(l[3].decode('utf-8')).lower()
        text=text.replace('"text":', '')
        
        #2)standardising slang words
        
        l1 = text.split(" ")
        if '' in l1:
            l1.remove('')
        for i in range(len(l1)):
            if l1[i] in slang_dict:
                l1[i] = slang_dict[l1[i]]
        text=' '.join(l1)
        
        #covert @user to at_user
        text= re.sub('@[^\s]+','atuser',text)
        
        #covert #data to data
        text = re.sub(r'#([^\s]+)', r'\1', text)
        
        #Remove additional white spaces
        text = re.sub('[\s]+', ' ', text)
        
        text  = text.replace('\\', '')
        #replace url by 'url'
        text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','url',text)
        
         #--feature vector--#

        #1)removing punctuation marks
        text =  text.translate(string.maketrans("",""), string.punctuation)
        
         #3) removing repeated chars
        pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
        text = pattern.sub(r"\1\1", text)
        
        #3) removing words containing no
        text=re.sub(r'\w*\d\w*', '', text).strip()
       
                
        
        sid = list(str(l[1]))[5:]
        sid = int(''.join(sid))
        
        return Row(tweet=str(text),tid=sid, status=0)
        
    else:
        return Row(tweet="not there",tid=0, status=0)
        
    

    


In [7]:
rdd1 = sc.textFile('/home/tanya/Desktop/major/after_phase1/dataset/dataset.txt')
rdd1.cache()
from pyspark.sql import Row

final = rdd1.map(preprocess)
final.cache()
df = sc.parallelize(final.collect())
df=df.toDF()
df=df.dropDuplicates(["tweet"])



In [ ]:

final1 = df.rdd
final1 = final1.filter(lambda x:x.tweet!="not there")
final1=final1.filter(lambda x:x.tweet!='')
#final1=final1.filter(lambda x:len(x.split(' ')>1))
final1.cache()
final1.count()
read_rdd = final1 ##read_id for mapping
read_rdd.take(4)
read_rdd.saveAsTextFile("/home/tanya/Desktop/major/after_phase1/analyse/classifiers/rdd_files/all_tweets.csv")


In [ ]:
def proc(x):
    status = x[0]
    sid = x[1]
    tweet = str(x[2])
    tweet = tweet.split(" ")
    tweet1 = [x for x in tweet if x not in stop_list ]
    tweet1=" ".join(tweet1)
    
    #spell correction
    skip = True
    suggestion_list = []
    distances = []
    data = []
    words=tweet1.split(" ")
    for k in range(len(words)):
        if ' ' in words:
            words.remove(' ')
    for k in range(len(words)):
        if  words[k]!='':


            if(d.check(words[k])==False):
                suggestion=d.suggest(words[k])
                if len(suggestion)!=0:
                    for i in range(0,len(suggestion)):
                        distance = editdistance.eval(words[k],suggestion[i])
                        distances.append(distance)
                        suggestion_list.append(suggestion[i])
                    index = distances.index(min(distances))
                    words[k]=suggestion_list[index]
        distances = []
        suggestion_list = []
    tweet1=" ".join(words)


    
    
    
    return Row(tid=sid,tweet=str(tweet1),status=status)
final1 = final1.map(proc)
final1.collect()


In [ ]:
#func to separate out abusive tweets
def process(x):
        for term in abusive:
            if term in x[2].split(" "):
                return x

In [ ]:
#create an abusive rdd from process func
abusive_tweets = final1.filter(process)
abusive_tweets.cache()
abusive_tweets.count()

In [ ]:
#write abusive tweets to file 
f = open("/home/tanya/Desktop/major/after_phase1/analyse/classifiers/rdd_files/abusive_tweets.csv","w")
for x in abusive_tweets.collect():
    for y in read_rdd.collect():
        if x[1] == y[1]:
            f.write(str(y))
            f.write('\n')
f.close()           

In [ ]:
refined_tweets = final1.subtract(abusive_tweets)
refined_tweets.cache()
refined_tweets.count()


In [ ]:
#function to filter out less offensive tweets
def filter_lessoffensive(x):
        for term in less_offensive:
            if term in x[2].split(" "):
                return x
            
            

In [ ]:
non_neutral = refined_tweets.filter(filter_lessoffensive)
non_neutral.count()

In [ ]:
#write nn tweets to file 
f = open("/home/tanya/Desktop/major/after_phase1/analyse/classifiers/rdd_files/nn_tweets.csv","w")
for x in non_neutral.collect():
    for y in read_rdd.collect():
        if x[1] == y[1]:
            f.write(str(y))
            f.write('\n')
f.close()           

In [ ]:
neutral_tweets =refined_tweets.subtract(non_neutral)    
#write neutral tweets to file 
f = open("/home/tanya/Desktop/major/after_phase1/analyse/classifiers/rdd_files/neutral_tweets.csv","w")
for x in neutral_tweets.collect():
    for y in read_rdd.collect():
        if x[1] == y[1]:
            f.write(str(y))
            f.write('\n')
f.close()
#neutral_tweets.collect()

In [50]:
neutral_tweets.count()

1885

In [ ]:
#########context based classification########


In [63]:
## after context eval rdd
def proc_change(x):
    l = str(x).split(",")
    #print l
    status = int((list(l[0])[11:][0]))
    #print status
    tid = int(l[1][5:])
    #print tid
    tweet = list(l[2])[9:]
    #print tweet
    tweet.pop(len(tweet)-1)
    tweet.pop(len(tweet)-1)
    tweet=''.join(tweet)
    return(Row(status=status,tid=tid,tweet=tweet))

neutral_tweets1 = sc.textFile('/home/tanya/Desktop/major/after_phase1/analyse/classifiers/rdd_files/neutral_tweets.csv')
neutral_tweets1=neutral_tweets1.map(proc_change)
neutral_tweets1.count()
    

1885

In [ ]:
####cosine similarity#####

In [9]:
#functions for calculating cosine similarity
word = re.compile(r'\w')
def cal_cosine(v1,v2):
    for  term in stop_list:
        if term in v1:
            del v1[term]
        if term in v2:
            del v2[term]
    set1 = v1.keys()
    set2 = v2.keys()
    num = sum([v1[x] * v2[x] for x in (set1 and set2)])
    
    prod1 = math.sqrt(sum(v1[x]**2 for x in v1.keys()))
    prod2 = math.sqrt(sum(v2[x]**2 for x in v2.keys()))
    den = prod1 * prod2
    
    if not den:
        return 
    else:
        return float(num) /den
    
def calc_vector(sentence):
    return Counter(sentence.split(" "))

##function to calculate the vector of all the phrases in dict
def calc_vector_phrase():
    phrase_vector=list()
    for phrase in phrase_list:
           phrase_vector.append(calc_vector(phrase))
    return phrase_vector

In [10]:
def filter_cosine(x):
    v1 = calc_vector(x[2])
    phrase_vector = calc_vector_phrase()
    for v2 in phrase_vector:
        cos = cal_cosine(v1,v2)
        if cos>0.8:
            return x

In [11]:
##phrase_cos_offensive is rdd containg the cosine rule offensive tweets
phrase_cos_offensive = neutral_tweets1.filter(filter_cosine)
phrase_cos_offensive.saveAsTextFile('/home/tanya/Desktop/major/after_phase1/analyse/classifiers/rdd_files/cos_offensive.csv')

15

In [13]:
# saving the clean tweets lleft after cosine in textfile
neutral_tweets1.subtract(phrase_cos_offensive).saveAsTextFile('/home/tanya/Desktop/major/after_phase1/analyse/classifiers/rdd_files/cos_clean.csv')

In [14]:

###classification based on adjectives

In [57]:


def filter_adjective(x):
    for term in adj :
        if term in x[2].split(" "):
            return x
    

In [58]:
adj = sorted(set(adj))
adjective_offensive = neutral_tweets1.filter(filter_adjective)
adjective_offensive.saveAsTextFile('/home/tanya/Desktop/major/after_phase1/analyse/classifiers/rdd_files/adj_off.csv')

1439

In [59]:
neutral_tweets1.subtract(adjective_offensive).saveAsTextFile('/home/tanya/Desktop/major/after_phase1/analyse/classifiers/rdd_files/adj_clean.csv')